In [ ]:
import time, os, ssl, time, queue, socket, struct, random, threading 
q = queue.Queue() # cache the command to be send

In [ ]:
def heartbeat_daemon():
# Send an invalid action to keep the connection alive
# would be ignored by rover but prevent a timeout for the control link
    while True: 
        q.put(struct.pack('<hi', -1, 0)) 
        time.sleep(3)
threading.Thread(target=heartbeat_daemon).start() 

In [ ]:
code2num = {    
      'ABS_X':  0 ,# 
      'ABS_Y':  1 ,# -32768~32767
     'ABS_RX':  2 ,# 
     'ABS_RY':  3 ,# 
      'ABS_Z':  4 ,# Break
     'ABS_RZ':  5 ,# Gas   0~255
  'ABS_HAT0Y':  6 ,# 
  'ABS_HAT0X':  7 ,# 
 'BTN_SELECT':  8 ,# 
  'BTN_START':  9 ,# 
  'BTN_SOUTH': 10 ,# A
   'BTN_EAST': 11 ,# B
  'BTN_NORTH': 12 ,# Y
   'BTN_WEST': 13 ,# X
     'BTN_TR': 14 ,# 
     'BTN_TL': 15 ,# 
}

In [ ]:
#!pip install inputs
import inputs # print(inputs.devices.gamepads) 
def controller_daemon(): 
    while True: 
        try:
            for event in inputs.get_gamepad():    # Events for Xbox One S Wireless Controller
                if event.ev_type in ['Key', 'Absolute']: 
                    q.put(struct.pack('<hi', code2num[event.code], event.state)) # put it into a queue
        except inputs.UnpluggedError as e:
            print(e)
            time.sleep(15)
threading.Thread(target=controller_daemon).start()

In [ ]:
# Establish connection with server

server_ip = 'yzlab3.chem.nyu.edu'
ctrl_port = 54322 
server_cert = 'secret/server.crt'
client_cert = 'secret/driver.crt'
client_key  = 'secret/driver.key'

context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key) 

def control_link_daemon():
    while True: 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
        ss.settimeout(5)  
        while True:
            try: 
                print('Try to connect the server')
                ss.connect((server_ip, ctrl_port))  
                print('Succeed a connection') 
                break
            except (ConnectionRefusedError, ConnectionResetError):
                print('Failed to connect the server and retry the connection in 5 seconds')
                time.sleep(5)
                continue
        while True:
            try:
                item = q.get()
                #print(struct.unpack('<hi', item))
                ss.send(item) # Send code and value to server  
            except socket.timeout as e: 
                ss.shutdown(socket.SHUT_RDWR)
                ss.close()
                break
            except (ConnectionAbortedError, ConnectionResetError, BrokenPipeError) as e:
                print('Server ended the connection and connection reestablishment will be retried in 5 seconds:', e)
                time.sleep(5) 
                break
threading.Thread(target=control_link_daemon).start() 